In [1]:
import sys, time
import numpy as np
import arkouda as ak

In [2]:
ak.connect(server="localhost", port=5555)

4.3.1
psp =  tcp://localhost:5555
connected to tcp://localhost:5555


In [3]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf"]

In [4]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start']

In [5]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

CPU times: user 1.72 ms, sys: 1.73 ms, total: 3.46 ms
Wall time: 6.17 s


In [6]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

[118781 118783 118785 ... 140369 140369 140369]
[34111590301 34111676798 48764590996 ... 34112191431 34112191431 34112191431]
[34111773101 34112055194 34111367844 ... 34111545319 31712083968 31712083968]
[5507 3137 5060 ... 78620 44270 49614]
[46272 445 5060 ... 95765 80 80]


In [7]:
print(nfDF)

{'srcIP': array([34111590301, 34111676798, 48764590996, ..., 34112191431, 34112191431, 34112191431]), 'dstIP': array([34111773101, 34112055194, 34111367844, ..., 34111545319, 31712083968, 31712083968]), 'srcPort': array([5507, 3137, 5060, ..., 78620, 44270, 49614]), 'dstPort': array([46272, 445, 5060, ..., 95765, 80, 80]), 'start': array([118781, 118783, 118785, ..., 140369, 140369, 140369])}


In [8]:
print(ak.info(ak.AllSymbols))

name:"id_5" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_4" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_3" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_2" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_1" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8



In [9]:
%time u,c = nfDF['srcIP'].unique(return_counts=True)

CPU times: user 1.32 ms, sys: 1.28 ms, total: 2.6 ms
Wall time: 13.8 s


In [10]:
print(u.size,u)
print(c.size,c)

25559 [48764978487 48764292094 34112110600 ... 48764444540 34111644622 34111308561]
25559 [176 180 3976 ... 183 561 2159]


In [11]:
%time u,c = nfDF['dstIP'].unique(return_counts=True)

CPU times: user 1.24 ms, sys: 1.51 ms, total: 2.74 ms
Wall time: 13.7 s


In [12]:
print(u.size,u)
print(c.size,c)

23790 [34111947526 34111337455 48764616254 ... 34111728417 48764965682 34112203985]
23790 [72 39 7 ... 8375 703 39]


In [13]:
%time u,c = nfDF['srcPort'].unique(return_counts=True)

CPU times: user 971 µs, sys: 1.57 ms, total: 2.54 ms
Wall time: 1.36 s


In [14]:
print(u.size,u)
print(c.size,c)

64899 [2 4 5 ... 99996 99997 99998]
64899 [188 223 2518 ... 284 220 220]


In [15]:
%time u,c = nfDF['dstPort'].unique(return_counts=True)

CPU times: user 1.24 ms, sys: 1.74 ms, total: 2.98 ms
Wall time: 1.33 s


In [16]:
print(u.size,u)
print(c.size,c)

59664 [2 4 7 ... 99996 99997 99998]
59664 [4 11 4698 ... 7 5 5]


In [17]:
print((nfDF['srcPort'] == 8080).sum())
print((nfDF['dstPort'] == 8080).sum())

2508
418986


In [21]:
s = u[0:64] # pick off some dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

[2 4 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 29 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 53 54 56 57 58 60 62 63 64 65 66 67 68 69 70 72 73 75 77]
CPU times: user 1.31 ms, sys: 5.97 ms, total: 7.28 ms
Wall time: 8.27 s
12272652


In [22]:
s = u[0:64].to_ndarray() # pick off some dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

[ 2  4  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 29
 30 31 32 33 34 35 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 53 54 56
 57 58 60 62 63 64 65 66 67 68 69 70 72 73 75 77]
CPU times: user 3.55 s, sys: 234 ms, total: 3.79 s
Wall time: 3.89 s
12272652


In [33]:
a = nfDF['start']
aMin = a.min()
aMax = a.max()
print(aMin,aMax, aMax - aMin)
h = ak.histogram(a,bins=100)
print(h)
print(h.sum(), a.size)

118781 140369 21588
[87 260 221 ... 507133 811672 769215]
60000000 60000000


In [20]:
#ak.shutdown()
#ak.disconnect()